<a href="https://colab.research.google.com/github/beyg1/Q4/blob/main/Openai%20SDK/ToolCallOpenaiAgent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -Uq openai-agents

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.4/100.4 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.7 MB/s eta 0:00:00


In [ ]:
import nest_asyncio
nest_asyncio.apply() #Make your Jupyter Notebook capable of running asynchronous functions.

In [ ]:
import os
from agents import Agent, Runner, AsyncOpenAI, OpenAIChatCompletionsModel
from agents.run import RunConfig
from google.colab import userdata


gemini_api_key = userdata.get("GOOGLE_API_KEY")


# Check if the API key is present; if not, raise an error
if not gemini_api_key:
    raise ValueError("GEMINI_API_KEY is not set. Please ensure it is defined in your .env file.")

#Reference: https://ai.google.dev/gemini-api/docs/openai
external_client = AsyncOpenAI(
    api_key=gemini_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)

model = OpenAIChatCompletionsModel(
    model="gemini-2.0-flash",
    openai_client=external_client
)

config = RunConfig(
    model=model,
    model_provider=external_client,
    tracing_disabled=True
    )

# Tools
The OpenAI Agents SDK provides a robust framework for integrating various tools into agents, enabling them to perform tasks such as data retrieval, web searches, and code execution. Here's an overview of the key points regarding tool integration:

# Types of Tools:

### Hosted Tools: These are pre-built tools running on OpenAI's servers, accessible via the [OpenAIResponsesModel]

### WebSearchTool: Enables agents to perform web searches.

### FileSearchTool: Allows retrieval of information from OpenAI Vector Stores.

### ComputerTool: Facilitates automation of computer-based tasks.

### Function Calling: This feature allows agents to utilize any Python function as a tool, enhancing their versatility.

### Agents as Tools: Agents can employ other agents as tools, enabling hierarchical task management without transferring control.

# Implementing Tools:

# Function Tools:
By decorating Python functions with @function_tool, they can be seamlessly integrated as tools for agents.

# Tool Execution Flow:
During an agent's operation, if a tool call is identified in the response, the SDK processes the tool call, appends the tool's response to the message history, and continues the loop until a final output is produced.

# Error Handling:
The SDK offers mechanisms to handle errors gracefully, allowing agents to recover from tool-related issues and continue their tasks effectively.

# Create Tools

In [ ]:
from agents.tool import function_tool
@function_tool("get_weather")
def get_weather(location: str) -> str:
  """
  Fetch the weather for a given location, returning a short description.
  """
  # Example logic
  return f"The weather in {location} is 22 degrees Celsius."

@function_tool("piaic_student_finder")
def student_finder(student_roll: int) -> str:
  """
  find the PIAIC student based on the roll number
  """
  data = {
          1: "Qasim",
          2: "Sir Zia",
          3: "Daniyal"
         }

  return data.get(student_roll, "Not Found")



In [ ]:
!pip install -Uq tavily-python
#tavily search provides agents with internet search

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 14.3 MB/s eta 0:00:00


In [ ]:
from agents.tool import function_tool
import os
from tavily import TavilyClient

# Retrieve the Tavily API key from environment variables or userdata
tavily_api_key = os.environ.get("Tavily_API_Key") or userdata.get("Tavily_API_Key")
if not tavily_api_key:
    raise ValueError("TAVILY_API_KEY is not set. Please ensure it is defined in your environment variables or userdata.")

# Initialize the Tavily search client
tavily_client = TavilyClient(api_key=tavily_api_key)

@function_tool("internet_search")
def internet_search(query: str) -> str:
  """
  Search the internet for the given query using Tavily and return the result.
  """
  response = tavily_client.search(query)

  # Assuming 'response' is a dictionary, extract the relevant information
  # and format it into a string.
  # You may need to adjust this based on the structure of Tavily's response.
  # Example:
  # return response.get('results', [])[0].get('content', '')
  return str(response) # For now, returning the entire response as a string



In [ ]:
import asyncio

from agents import Agent, Runner


async def main():
    agent = Agent(
        name="Assistant",
        instructions="You only respond in haikus.",
        tools=[get_weather, student_finder, internet_search], # add tools here
        model=model
    )

    result = await Runner.run(agent, "can you access the internet search tool and find out who is the president of Pakistan")
    print(result.final_output)



if __name__ == "__main__":
    asyncio.run(main())

Asif Ali Zardari
Is president now, it is he,
Takes the office now.

